In [1]:
from datasets import load_metric
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

import pandas as pd
import os
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer
import soundfile as sf
import os
# Use old or new vocab? 
os.system("cp vocab_300_with_numbers.json vocab.json")



tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
        "/home/peterr/macocu/task8/", unk_token="[UNK]", 
        pad_token="[PAD]", word_delimiter_token=" ")
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)
model = Wav2Vec2ForCTC.from_pretrained("/home/peterr/macocu/task8/31_normalised_220k/checkpoint-6500").cuda()

def transcribe(path):
    speech, sample_rate = sf.read(path)
    input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()

    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]).lower()
    return transcription


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
import pandas as pd

df = pd.read_csv("35_eval_split.csv")
df.columns

Index(['hashname', 'orig_file', 'start', 'end', 'words', 'word_start_times',
       'norm_words', 'norm_words_start_times', 'utterance_id_start',
       'utterance_id_end', 'speaker_info', 'split', 'norm_words_edited',
       'sentence', 'Speaker_role', 'Speaker_type', 'Speaker_party',
       'Speaker_party_name', 'Party_status', 'Speaker_name', 'Speaker_gender',
       'Speaker_birth'],
      dtype='object')

In [4]:
df["original_transcript"] = df.words.apply(" ".join)

In [5]:
import os
df["path"] = df.hashname.apply(lambda s: os.path.join("/home/peterr/macocu/task8/transfer", s))
df["model_transcript"] = df.path.apply(transcribe)

In [6]:
df.to_csv("36_eval_split_with_full_data_and_transcription.csv", index=False)